In [3]:
def account_acc_hier(filename_suffix):
    acc_matrix = []
    for i in range(5):
        concGothr = []
        for j in range(2):
            with open('./'+filename_suffix+str(i)+str(j)+'.out','r') as f:
                best_val_acc = 0.0
                last_test_acc = 0.0
                test_record_flag = False
                info = f.readlines()
                for line in info:
                    if line.startswith('val set accuracy:'):
                        tokens = line.split()
                        val_acc = float(tokens[-2])
                        if val_acc >= best_val_acc:
                            best_val_acc = val_acc
                            test_record_flag = True

                    if line.startswith('test set accuracy:'):
                        tokens = line.split()
                        test_acc = float(tokens[-2])
                        if test_record_flag:
                            last_test_acc = test_acc
                            test_record_flag = False

            concGothr.append((best_val_acc, last_test_acc))
        acc_matrix.append(concGothr)
    return acc_matrix

def account_acc(filename_suffix):
    acc_matrix = []
    for i in range(5):
        with open('./'+filename_suffix+str(i)+'.out','r') as f:
            best_val_acc = 0.0
            last_test_acc = 0.0
            test_record_flag = False
            info = f.readlines()
            for line in info:
                if line.startswith('val set accuracy:'):
                    tokens = line.split()
                    val_acc = float(tokens[-2])
                    if val_acc >= best_val_acc:
                        best_val_acc = val_acc
                        test_record_flag = True

                if line.startswith('test set accuracy:'):
                    tokens = line.split()
                    test_acc = float(tokens[-2])
                    if test_record_flag:
                        last_test_acc = test_acc
                        test_record_flag = False

            acc_matrix.append((best_val_acc, last_test_acc))
    return acc_matrix


In [23]:
atthier_acc = account_acc_hier('ATTCVHier')

atthier_acc[1][1] = (0.0, 0.0)
atthier_acc[2][0] = (0.0, 0.0)

atthier_acc

[[(81.6711590296, 70.6039551042), (80.0539083558, 71.0849812934)],
 [(80.5929919137, 72.528059861), (0.0, 0.0)],
 [(0.0, 0.0), (82.4797843666, 70.5505077499)],
 [(80.0539083558, 73.4366648851), (79.5148247978, 72.1004810262)],
 [(77.358490566, 70.4970603955), (78.4366576819, 71.9935863175)]]

In [4]:
nonatthier_acc = account_acc_hier('NonAttHier')
nonatthier_acc

[[(81.6711590296, 72.4211651523), (80.3234501348, 70.2298236237)],
 [(80.5929919137, 71.2453233565), (82.2102425876, 71.1918760021)],
 [(81.4016172507, 72.528059861), (81.4016172507, 70.1763762694)],
 [(80.8625336927, 69.214323891), (81.9407008086, 74.7194013896)],
 [(77.0889487871, 72.8487439872), (76.5498652291, 69.6953500802)]]

In [24]:
import numpy as np

ave_concate_hier = np.sum(np.asarray([atthier_acc[i][0][0] for i in range(5)]))/4.0
ave_gothrough_hier = np.sum(np.asarray([atthier_acc[i][1][0] for i in range(5)]))/4.0

print ave_concate_hier
print ave_gothrough_hier

79.9191374663
80.1212938005


In [10]:

nonatt_ave_concate_hier = np.sum(np.asarray([nonatthier_acc[i][0][0] for i in range(5)]))/5.0
nonatt_ave_gothrough_hier = np.sum(np.asarray([nonatthier_acc[i][1][0] for i in range(5)]))/5.0

print nonatt_ave_concate_hier
print nonatt_ave_gothrough_hier

80.3234501348
80.4851752022


In [13]:
val_test_pair = account_acc('PlaintEnd')

In [14]:
val_acc_ls = np.asarray([val_test_pair[i][0] for i in range(5)])
ave_acc = sum(val_acc_ls)/ 5.0
print ave_acc

76.4420485175


In [13]:
import cPickle as pickle
import numpy as np

with open('../../data/pickles/test_index_corpus.pkl', 'r') as f:
    test_set = pickle.load(f)
    test_ans = test_set[3]
test_ans_vec = np.asarray(test_ans)

In [17]:

five_scores = []
for i in range(5):
    print str(i)
    f = open('./NonAttHier_record_matrixsequence-gothrough-DNNhinge-150x75x1-1-0.0-'+str(i)+'.pkl', 'r')
    a = pickle.load(f)
    f.close()
    
    five_scores.append(a)
end1_scores = [five_scores[i][:,0] for i in range(5)]
end2_scores = [five_scores[i][:,1] for i in range(5)]

def concate(end_scores):
    end_reshape = [end1.reshape((-1,1)) for end1 in end_scores]
    end_concate = np.concatenate(end_reshape, axis = 1)
    return end_concate

end1_matrix = concate(end1_scores)
end2_matrix = concate(end2_scores)

final_end1_score = end1_matrix.sum(axis = 1).reshape((-1,1))
final_end2_score = end2_matrix.sum(axis = 1).reshape((-1,1))
final_decision = np.argmax(np.concatenate((final_end1_score,final_end2_score), axis = 1),axis = 1)

acc = (1871.0 - np.count_nonzero((final_decision - test_ans_vec)))/ 1871.0
print acc

0
1
2
3
4
0.721539283805


In [86]:
end1_scores = [five_scores[i][:,0] for i in range(5)]
end2_scores = [five_scores[i][:,1] for i in range(5)]

In [87]:
import numpy as np
def concate(end_scores):
    end_reshape = [end1.reshape((-1,1)) for end1 in end_scores]
    end_concate = np.concatenate(end_reshape, axis = 1)
    return end_concate

end1_matrix = concate(end1_scores)
end2_matrix = concate(end2_scores)


In [88]:
final_end1_score = end1_matrix.sum(axis = 1).reshape((-1,1))
final_end2_score = end2_matrix.sum(axis = 1).reshape((-1,1))
final_decision = np.argmax(np.concatenate((final_end1_score,final_end2_score), axis = 1),axis = 1)


In [3]:
import cPickle as pickle
import numpy as np
with open('../../data/pickles/test_index_corpus.pkl', 'r') as f:
    test_set = pickle.load(f)
    test_ans = test_set[3]
test_ans_vec = np.asarray(test_ans)

In [90]:
acc = (1871.0 - np.count_nonzero((final_decision - test_ans_vec)))/ 1871.0


In [91]:
print acc

0.663281667557


In [128]:
with open('./test_score_record_matrix_plaint_endlast--hinge-512x1-1-0.0-000.pkl','r') as f:
    test_scores = pickle.load(f)


In [129]:
predict_answer = np.argmax(test_scores, axis = 1)
correct_label = np.ones(1871)
for i in xrange(1871):
    if predict_answer[i] != test_ans_vec[i]:
        correct_label[i] = 0
        

In [130]:
acc = np.sum(correct_label)/1871.0
print acc

0.697487974345


In [131]:
# retrieve correct classified scores 
right_classify_scores = []
faulse_classify_scores = []
for i in range(1871):
    if predict_answer[i] == test_ans_vec[i]:
        right_classify_scores.append([i,test_scores[i]])
    else:
        faulse_classify_scores.append([i,test_scores[i]])

In [132]:
index_vec = np.arange(1871).reshape((1871, 1))
answer_vec = test_ans_vec.reshape((1871, 1))
predic_vec = predict_answer.reshape((1871, 1))
correct_vec = correct_label.reshape((1871,1))

In [133]:
big_score_matrix = np.concatenate([index_vec, test_scores, predic_vec, answer_vec, correct_vec], axis = 1)

In [134]:
print big_score_matrix[:10]

[[ 0.         -0.02268939  0.26555426  1.          1.          1.        ]
 [ 1.          0.36979881  0.60613192  1.          0.          0.        ]
 [ 2.          0.60118569 -0.92172158  0.          1.          0.        ]
 [ 3.          0.7622604   0.99312969  1.          1.          1.        ]
 [ 4.          0.97404977 -0.38604876  0.          0.          1.        ]
 [ 5.         -0.53695388 -0.29275038  1.          0.          0.        ]
 [ 6.          0.5844534   0.88971448  1.          0.          0.        ]
 [ 7.         -0.26619499  0.99529875  1.          1.          1.        ]
 [ 8.         -0.4218886   0.90821223  1.          1.          1.        ]
 [ 9.          0.45760338  0.42673224  0.          1.          0.        ]]


In [135]:
single_score_end_matrix = np.zeros((1871*2, 3))
for i in range(1871):
    single_score_end_matrix[i*2][0] = i
    single_score_end_matrix[i*2+1][0] = i+0.1
    single_score_end_matrix[i*2][1] = big_score_matrix[i][1]
    single_score_end_matrix[i*2+1][1] = big_score_matrix[i][2]
    single_score_end_matrix[i*2][2] = correct_label[i]
    single_score_end_matrix[i*2+1][2] = correct_label[i]

In [136]:
print single_score_end_matrix[:100]

[[  0.00000000e+00  -2.26893908e-02   1.00000000e+00]
 [  1.00000000e-01   2.65554260e-01   1.00000000e+00]
 [  1.00000000e+00   3.69798808e-01   0.00000000e+00]
 [  1.10000000e+00   6.06131915e-01   0.00000000e+00]
 [  2.00000000e+00   6.01185687e-01   0.00000000e+00]
 [  2.10000000e+00  -9.21721578e-01   0.00000000e+00]
 [  3.00000000e+00   7.62260402e-01   1.00000000e+00]
 [  3.10000000e+00   9.93129686e-01   1.00000000e+00]
 [  4.00000000e+00   9.74049769e-01   1.00000000e+00]
 [  4.10000000e+00  -3.86048757e-01   1.00000000e+00]
 [  5.00000000e+00  -5.36953880e-01   0.00000000e+00]
 [  5.10000000e+00  -2.92750378e-01   0.00000000e+00]
 [  6.00000000e+00   5.84453405e-01   0.00000000e+00]
 [  6.10000000e+00   8.89714479e-01   0.00000000e+00]
 [  7.00000000e+00  -2.66194994e-01   1.00000000e+00]
 [  7.10000000e+00   9.95298753e-01   1.00000000e+00]
 [  8.00000000e+00  -4.21888601e-01   1.00000000e+00]
 [  8.10000000e+00   9.08212229e-01   1.00000000e+00]
 [  9.00000000e+00   4.57603

In [137]:
sorted_score_matrix = single_score_end_matrix[(-single_score_end_matrix[:,1]).argsort()]

In [138]:
sorted_score_matrix[:100]

array([[  6.49100000e+02,   9.99999992e-01,   1.00000000e+00],
       [  8.81000000e+02,   9.99999962e-01,   1.00000000e+00],
       [  1.78400000e+03,   9.99999959e-01,   1.00000000e+00],
       [  1.38800000e+03,   9.99999956e-01,   1.00000000e+00],
       [  1.31700000e+03,   9.99999919e-01,   0.00000000e+00],
       [  8.41000000e+02,   9.99999902e-01,   1.00000000e+00],
       [  3.08000000e+02,   9.99999899e-01,   1.00000000e+00],
       [  1.94000000e+02,   9.99999892e-01,   1.00000000e+00],
       [  8.10100000e+02,   9.99999884e-01,   1.00000000e+00],
       [  1.55700000e+03,   9.99999849e-01,   1.00000000e+00],
       [  1.39310000e+03,   9.99999820e-01,   1.00000000e+00],
       [  4.14000000e+02,   9.99999717e-01,   1.00000000e+00],
       [  1.54310000e+03,   9.99999679e-01,   0.00000000e+00],
       [  1.18500000e+03,   9.99999670e-01,   1.00000000e+00],
       [  1.81000000e+02,   9.99999651e-01,   1.00000000e+00],
       [  3.47100000e+02,   9.99999642e-01,   0.0000000

In [155]:
end1 = test_set[1]
end2 = test_set[2]

with open('../../data/pickles/index2word_dict.pkl') as f:
    index2word_dict = pickle.load(f)
def index2word(sent):
    sent_text = []
    for index in sent:
        sent_text.append(index2word_dict[index])
    sent_str = " ".join(sent_text)
    return sent_str

#show the top 50 scoring endings
print_out_num = 100
correct_count = 0
ave_len = 0.0
for i in xrange(print_out_num):
    index = int(sorted_score_matrix[i][0])
    ends = int(sorted_score_matrix[i][0]*10) % 10
    if ends == 0:
        end_text = end1[index]
    else:
        end_text = end2[index]
    if int(sorted_score_matrix[i][2]) == 1:
        correct_count += 1
    ave_len += len(end_text)
    print index2word(end_text), sorted_score_matrix[i][1] , sorted_score_matrix[i][2]
print correct_count / float(print_out_num)
print ave_len / print_out_num
#show the bottom 50 scoring endings

<s> a couple of days later , i started to feel better , thank god ! </s> 0.999999991781 1.0
<s> the man quickly tried to round them all up , but it was difficult . </s> 0.999999961746 1.0
<s> she got a good grade on the report and raised her math grade . </s> 0.999999959422 1.0
<s> her friends had been trying to tell gina they 'd had a baby ! </s> 0.999999955667 1.0
<s> i feel like i 'm living a dream come true , thanks to chicago ! </s> 0.999999918673 0.0
<s> it created such a mess , maybe it was n't worth saving the money . </s> 0.999999901555 1.0
<s> june cried herself to sleep and woke up hungry in the morning . </s> 0.9999998992 1.0
<s> she saved enough money to buy a cell phone on her own . </s> 0.999999892225 1.0
<s> sue was upset to learn that she had messed up the recipe . </s> 0.999999883771 1.0
<s> he tried out for the team again and this time , he made it . </s> 0.999999848958 1.0
<s> the speaker was able to give her all of the answers she needed . </s> 0.999999819626 1.0
<

In [156]:
#show the top 50 scoring endings
print_out_num = 100
correct_count = 0
ave_len = 0.0
for i in xrange(print_out_num):
    index = int(sorted_score_matrix[1870-(print_out_num-i)][0])
    ends = int(sorted_score_matrix[1870-(print_out_num - i)][0]*10) % 10
    if ends == 0:
        end_text = end1[index]
    else:
        end_text = end2[index]
    if int(sorted_score_matrix[1870 - (print_out_num - i)][2]) == 1:
        correct_count += 1
    ave_len += len(end_text)
    print index2word(end_text), sorted_score_matrix[1870 -(print_out_num -i)][1] , sorted_score_matrix[1870-(print_out_num - i)][2]
print correct_count / float(print_out_num)
print ave_len/print_out_num


<s> he had caught an old boot . </s> 0.630430510838 1.0
<s> he soon had to find a house to move to . </s> 0.630266157442 1.0
<s> jill was not well rehearsed enough and performed poorly . </s> 0.63011840427 0.0
<s> i had run out of milk . </s> 0.629574642911 1.0
<s> he has to go to the store on sundays for new socks . </s> 0.629041139105 1.0
<s> the baby then crawled around the home . </s> 0.628552382892 1.0
<s> jill and her dad took both kittens home . </s> 0.628132420553 0.0
<s> charles lost interest in sherry when she became self supporting . </s> 0.627498638387 1.0
<s> the teacher yelled at the girl who had tripped . </s> 0.627078205419 1.0
<s> felicia expected to lose her job the next day . </s> 0.626856741204 1.0
<s> the poor puppy was hit by a car . </s> 0.626850691851 1.0
<s> the whole family enjoyed the experience . </s> 0.626744776444 1.0
<s> jim was angry . </s> 0.626220344988 1.0
<s> amy then purchased them from the store . </s> 0.625714649916 1.0
<s> miles got up and walked